In [1]:
from get_data import get_structured_data
import utility
from classifiers import *
from sklearn.model_selection import train_test_split
from constants import RANDOM_STATE, CV, MIN_NUM_PRICES
from sklearn.model_selection import GridSearchCV
from island_week_data import TurnipPattern

In [2]:
rows = get_structured_data()
all_X, _, all_y = utility.get_all_data(rows)
all_X.shape, all_y.shape

((83, 12), (83, 1))

In [3]:
rows = get_structured_data()
all_X, _, all_y = utility.get_all_data(rows)
all_X.shape, all_y.shape

((83, 12), (83, 1))

In [4]:
perfect_x, _, perfect_y = utility.get_perfect_data(rows)
perfect_x.shape, perfect_y.shape

((25, 12), (25, 1))

In [5]:
classifiers = [get_linear_svm_classifier(), get_rbf_svm_classifier(), get_naive_bayes_classifier(), get_random_forest_classifier()]

In [6]:
# TODO: Copy the ROC stuff from Plotter to this cell
# then save the sensitivity and specificity like you did in HW3
# Taken from my HW 3
def get_roc_stuff(prediction, truth):
    n_classes = len([tp for tp in TurnipPattern])
    # Shoutout to this person for helping me with this.
    # https://kawahara.ca/how-to-compute-truefalse-positives-and-truefalse-negatives-in-python-for-binary-classification-problems/
    true_positives = np.nan_to_num(np.asarray([np.logical_and(prediction == i, truth == i) 
                                               for i in range(n_classes)])).sum(axis=1)
    false_positives = np.nan_to_num(np.asarray([np.logical_and(prediction == i, truth != i) 
                                                for i in range(n_classes)])).sum(axis=1)
    true_negatives = np.nan_to_num(np.asarray([np.logical_and(prediction != i, truth != i) 
                                               for i in range(n_classes)])).sum(axis=1)
    false_negatives = np.nan_to_num(np.asarray([np.logical_and(prediction != i, truth == i) 
                                                for i in range(n_classes)])).sum(axis=1)
    sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))
    specificity = np.nan_to_num(true_negatives / (true_negatives + false_negatives))
    return sensitivity, specificity

In [7]:
def train_classifier(class_name, classifier, params, train_x, train_y, test_x, test_y, cv, save_model=True):
    print(f'Optimizing parameters...')
    param_keys = '\n'.join([k for k in params.keys()])
    print(f'List of parameters: \n{param_keys}')
    grid = GridSearchCV(classifier, params, cv=cv, n_jobs=N_JOBS)
    grid.fit(train_x, train_y)
    model = grid.best_estimator_
    print(f'Optimal Parameters Found to be: ')
    print('\n'.join([f'{k}: {v}' for k, v in grid.best_params_.items()]))
    
    model.fit(train_x, train_y)
    
    print(f'Testing Classifier with {test_x.shape[0]} samples.')
    y_pred = model.predict(test_x)
    n_tests = test_y.shape[0]
    num_wrong = (y_pred != test_y).sum()
    
    if save_model:
        filepath, _ = utility.save_model(class_name, model)
        print(f'Model was saved to: {filepath}')

    full_X = np.append(train_x, test_x, axis=0)
    full_y = np.append(train_y, test_y, axis=0)
    
    pred_y = model.predict(full_X)
    sens, spec = get_roc_stuff(pred_y, full_y)
        
    print(f'Got {num_wrong}/{n_tests} ({num_wrong / n_tests:.2%}) tests wrong for {class_name}.')
    
    return num_wrong, n_tests, np.average(sens), np.average(spec)

In [8]:
def train_all_classifiers(X, y, train_percent, save_models=True):
    cv = 1
    n_iterations = 0
    while cv == 1:
        train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=train_percent, random_state=RANDOM_STATE + n_iterations)
        cv = utility.get_max_cv(train_y)
        n_iterations += 1
    print(f'Using CV: {cv}')
    history = []
    
    train_y = train_y.reshape(-1)
    test_y = test_y.reshape(-1)
    
    for class_name, classifier, params in classifiers:
        print(f'Training/Testing {class_name}')
        num_wrong, n_tests, sensitivity, specificity = train_classifier(class_name, classifier, params, train_x, train_y, test_x, test_y, cv, save_model=save_models)
        best_params = {k: v for k, v in classifier.get_params().items() if k in params.keys()}
        last_result = {'classifier': class_name, 'min_num_prices': MIN_NUM_PRICES, 'sensitivity': sensitivity, 'specificity': specificity, 'n_wrong': int(num_wrong), 'n_tests': int(n_tests), 'best_params': best_params, 'train_size': float(train_percent)}
        history.append(last_result)
        print('')
    print('\n'.join(['\n'.join([f'{k}: {v}' for k, v in result.items()]) for result in history]))
    return history

In [9]:
training_sizes = np.arange(.1, .91, .1)
results = []
for training_size in training_sizes:
    curr_results = train_all_classifiers(all_X, all_y, training_size)
    results.extend(curr_results)
    print('')

utility.save_results(results)

Using CV: 2
Training/Testing Linear SVM
Optimizing parameters...
List of parameters: 
linearsvc__C
linearsvc__fit_intercept
linearsvc__tol
Optimal Parameters Found to be: 
linearsvc__C: 0.22499999999999998
linearsvc__fit_intercept: False
linearsvc__tol: 1e-06
Testing Classifier with 75 samples.
Model was saved to: ./models/LinearSVM_20201123_194910.mdl
Got 36/75 (48.00%) tests wrong for Linear SVM.

Training/Testing RBF SVM
Optimizing parameters...
List of parameters: 
svc__gamma
svc__shrinking
svc__tol
svc__C


<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))


Optimal Parameters Found to be: 
svc__C: 0.125
svc__gamma: scale
svc__shrinking: True
svc__tol: 1e-05
Testing Classifier with 75 samples.
Model was saved to: ./models/RBFSVM_20201123_194911.mdl
Got 37/75 (49.33%) tests wrong for RBF SVM.

Training/Testing Naive Bayes
Optimizing parameters...
List of parameters: 
var_smoothing
Optimal Parameters Found to be: 
var_smoothing: 1e-12
Testing Classifier with 75 samples.
Model was saved to: ./models/NaiveBayes_20201123_194911.mdl
Got 39/75 (52.00%) tests wrong for Naive Bayes.

Training/Testing Random Forest
Optimizing parameters...
List of parameters: 
ccp_alpha
max_features
max_depth
n_estimators


<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))
<ipython-input-6-e6060f77009d>:17: RuntimeWarning: invalid value encountered in true_divide
  specificity = np.nan_to_num(true_negatives / (true_negatives + false_negatives))
<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))


Optimal Parameters Found to be: 
ccp_alpha: 0.01
max_depth: None
max_features: auto
n_estimators: 50
Testing Classifier with 75 samples.
Model was saved to: ./models/RandomForest_20201123_195104.mdl
Got 39/75 (52.00%) tests wrong for Random Forest.

classifier: Linear SVM
min_num_prices: 11
sensitivity: 0.25609756097560976
specificity: 0.9274169850132236
n_wrong: 36
n_tests: 75
best_params: {'linearsvc__C': 1.0, 'linearsvc__fit_intercept': True, 'linearsvc__tol': 0.0001}
train_size: 0.1
classifier: RBF SVM
min_num_prices: 11
sensitivity: 0.08835341365461848
specificity: 0.7590361445783133
n_wrong: 37
n_tests: 75
best_params: {'svc__C': 1.0, 'svc__gamma': 'scale', 'svc__shrinking': True, 'svc__tol': 0.001}
train_size: 0.1
classifier: Naive Bayes
min_num_prices: 11
sensitivity: 0.15914786967418545
specificity: 0.8346393091161639
n_wrong: 39
n_tests: 75
best_params: {'var_smoothing': 1e-09}
train_size: 0.1
classifier: Random Forest
min_num_prices: 11
sensitivity: 0.17009320175438594
speci

...

<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))
<ipython-input-6-e6060f77009d>:17: RuntimeWarning: invalid value encountered in true_divide
  specificity = np.nan_to_num(true_negatives / (true_negatives + false_negatives))


Optimal Parameters Found to be: 
svc__C: 0.125
svc__gamma: scale
svc__shrinking: True
svc__tol: 1e-05
Testing Classifier with 9 samples.
Model was saved to: ./models/RBFSVM_20201123_201052.mdl
Got 6/9 (66.67%) tests wrong for RBF SVM.

Training/Testing Naive Bayes
Optimizing parameters...
List of parameters: 
var_smoothing
Optimal Parameters Found to be: 
var_smoothing: 1e-12
Testing Classifier with 9 samples.
Model was saved to: ./models/NaiveBayes_20201123_201052.mdl
Got 4/9 (44.44%) tests wrong for Naive Bayes.

Training/Testing Random Forest
Optimizing parameters...
List of parameters: 
ccp_alpha
max_features
max_depth
n_estimators


<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))
<ipython-input-6-e6060f77009d>:17: RuntimeWarning: invalid value encountered in true_divide
  specificity = np.nan_to_num(true_negatives / (true_negatives + false_negatives))
<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))


Optimal Parameters Found to be: 
ccp_alpha: 0.01
max_depth: None
max_features: None
n_estimators: 140
Testing Classifier with 9 samples.
Model was saved to: ./models/RandomForest_20201123_201248.mdl
Got 5/9 (55.56%) tests wrong for Random Forest.

classifier: Linear SVM
min_num_prices: 11
sensitivity: 0.08835341365461848
specificity: 0.7590361445783133
n_wrong: 6
n_tests: 9
best_params: {'linearsvc__C': 1.0, 'linearsvc__fit_intercept': True, 'linearsvc__tol': 0.0001}
train_size: 0.9
classifier: RBF SVM
min_num_prices: 11
sensitivity: 0.08835341365461848
specificity: 0.7590361445783133
n_wrong: 6
n_tests: 9
best_params: {'svc__C': 1.0, 'svc__gamma': 'scale', 'svc__shrinking': True, 'svc__tol': 0.001}
train_size: 0.9
classifier: Naive Bayes
min_num_prices: 11
sensitivity: 0.47345679012345676
specificity: 0.9066793784602004
n_wrong: 4
n_tests: 9
best_params: {'var_smoothing': 1e-09}
train_size: 0.9
classifier: Random Forest
min_num_prices: 11
sensitivity: 0.8072441742654508
specificity: 0

<ipython-input-6-e6060f77009d>:16: RuntimeWarning: invalid value encountered in true_divide
  sensitivity = np.nan_to_num(true_positives / (true_positives + false_positives))
